# Computes trends of time series by fitting a low order polinomial

WARNING: This component currently only supports local execution (not Kubeflow/Airflow)  
WARNING: This component currently only supports copernicus climate data


Future work    
[ ] Generalize component  
[ ] Make component run on KubeFlow/Airflow pipelines

In [ ]:
# @param data_dir temporal data storage for local execution
# @param data_parquet path and parquet file name (default: data.parquet)
# @param master url of master (default: local mode)

In [ ]:
!pip3 install pyspark==3.1.1

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import pickle
import numpy as np

In [ ]:
data_parquet = os.environ.get('data_parquet', 'data.parquet')
master = os.environ.get('master', "local[*]")
data_dir = os.environ.get('data_dir', '../../data/')

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster(master))
spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.read.parquet(data_dir+data_parquet)

In [ ]:
df.createOrReplaceTempView('df')

In [ ]:
if os.path.exists(data_dir+'lon_lat.p'):
    lon_lat = pickle.load(open(data_dir+"lon_lat.p", "rb"))
else:
    lon_lat = spark.sql('''
        select lon, lat from df where group by lon, lat
    ''').rdd.map(lambda x: [x.lon, x.lat]).collect()
    pickle.dump(lon_lat, open(data_dir+"lon_lat.p", "wb" ))

In [ ]:
if os.path.exists(data_dir+'lon_lat_slope.p'):
    lon_lat_slope = pickle.load(open(data_dir+"lon_lat_slope.p", "rb"))
else:
    lon_lat_slope = dict()

In [ ]:
for index in range(len(lon_lat)):
    if not lon_lat[index][0]+lon_lat[index][1] in lon_lat_slope:
        sm = spark.sql("select sm from df where sm <> 'null' and lat='"+
                       lon_lat[index][0]+
                       "' and lon='"+
                       lon_lat[index][1]+
                       "' order by time asc").collect()
        import numpy as np
        sm = np.array(list(map(lambda x :x.sm, sm)))
        sm = sm.astype(float)
        if len(sm)> 0:
            coefficients, residuals, _, _, _ = np.polyfit(range(len(sm)),sm,1,full=True)
            mse = residuals[0]/(len(sm))
            nrmse = np.sqrt(mse)/(sm.max() - sm.min())
            print('Slope ' + str(coefficients[1]))
            print('NRMSE: ' + str(nrmse))
            lon_lat_slope[lon_lat[index][0]+lon_lat[index][1]]=[coefficients[1]]
            pickle.dump(lon_lat_slope, open(data_dir+"lon_lat_slope.p", "wb"))
        else:
            print('skipping, empty data')
    else:
            print('already processed')